In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from os import listdir
import shutil
from shutil import copyfile
import random
import zipfile

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications.vgg16 import VGG16


# Dataset

## Define the training and validation base directories

In [2]:
# Define root directory
root_dir = './'


def create_train_test_dirs(root_path):
  ### START CODE HERE
  os.makedirs(os.path.join(root_path, 'training'))
  os.makedirs(os.path.join(root_path, 'testing'))
  os.makedirs(os.path.join(root_path, 'training/0'))
  os.makedirs(os.path.join(root_path, 'training/1'))
  os.makedirs(os.path.join(root_path, 'testing/0'))
  os.makedirs(os.path.join(root_path, 'testing/1'))
  
try:
  create_train_test_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [3]:
# Define root directory
os.makedirs(os.path.join('./', 'IDC_regular_ps50_idx5'))
root_dir = './IDC_regular_ps50_idx5/'


def create_train_test_dirs(root_path):
  ### START CODE HERE
  
  os.makedirs(os.path.join(root_path, 'training'))
  os.makedirs(os.path.join(root_path, 'testing'))
  os.makedirs(os.path.join(root_path, 'training/0'))
  os.makedirs(os.path.join(root_path, 'training/1'))
  os.makedirs(os.path.join(root_path, 'testing/0'))
  os.makedirs(os.path.join(root_path, 'testing/1'))
  
try:
  create_train_test_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [4]:
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

./IDC_regular_ps50_idx5/training
./IDC_regular_ps50_idx5/testing
./IDC_regular_ps50_idx5/training/0
./IDC_regular_ps50_idx5/training/1
./IDC_regular_ps50_idx5/testing/0
./IDC_regular_ps50_idx5/testing/1


## Split the data

In [13]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE, all_files, d, counter):
    c = 0
    for name in all_files:
      if name == '.DS_Store':
        continue
      if c==counter: break
      
    # d = False --> choose 0 folders(../input/breast-histopathology-images/IDC_regular_ps50_idx5/10295/0) 
      if d == False:
        name+='/0'
    # d = False --> choose 1 folders(../input/breast-histopathology-images/IDC_regular_ps50_idx5/10295/1)
      else:
        name+='/1'

      files = []
      for i in os.listdir(SOURCE + name + '/'):
        # img = image path
        img = SOURCE + name + '/' + i

        if os.path.getsize(img) != 0:
          files.append(img)
        else:
          print(f"{img} is zero length, so ignoring")
    # Shuffle the image if the list
      random.shuffle(files)
      length = int(round(len(files) * SPLIT_SIZE, 0))
    # Split the data
      train_files= files[0:length]
      test_files = files[length:]
      for i in train_files:
        x = i.split('/')
        copyfile(i, TRAINING + x[-1])

      for i in test_files:
        x = i.split('/')
        copyfile(i, TESTING + x[-1])
    #         print('Copy', i)
#       print(name, 'Done')
      c+=1  

In [14]:
all_files = listdir("/Users/sissi/Desktop/564biiig/input/breast-histopathology-images/IDC_regular_ps50_idx5")
len(all_files)

280

In [15]:
# Test your split_data function

# Define paths
false_SOURCE_DIR = "/Users/sissi/Desktop/564biiig/input/breast-histopathology-images/IDC_regular_ps50_idx5/"
true_SOURCE_DIR = "/Users/sissi/Desktop/564biiig/input/breast-histopathology-images/IDC_regular_ps50_idx5/"

TRAINING_DIR = "./IDC_regular_ps50_idx5/training/"
TESTING_DIR = "./IDC_regular_ps50_idx5/testing/"

TRAINING_FALSE_DIR = os.path.join(TRAINING_DIR, "0/")
TESTING_FALSE_DIR = os.path.join(TESTING_DIR, "0/")

TRAINING_TRUE_DIR = os.path.join(TRAINING_DIR, "1/")
TESTING_TRUE_DIR = os.path.join(TESTING_DIR, "1/")

# Empty directories in case you run this cell multiple times
if len(os.listdir(TRAINING_FALSE_DIR)) > 0:
  for file in os.scandir(TRAINING_FALSE_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_TRUE_DIR)) > 0:
  for file in os.scandir(TRAINING_TRUE_DIR):
    os.remove(file.path)
if len(os.listdir(TESTING_FALSE_DIR)) > 0:
  for file in os.scandir(TESTING_FALSE_DIR):
    os.remove(file.path)
if len(os.listdir(TESTING_TRUE_DIR)) > 0:
  for file in os.scandir(TESTING_TRUE_DIR):
    os.remove(file.path)

# Define proportion of images used for training
split_size = .9

# Define num of folders for training(will go for all folders if it is +275)
# Usefull to test your model with small data first then use all data.
counter = 300
# Run the function
# NOTE: Messages about zero length images should be printed out
split_data(false_SOURCE_DIR, TRAINING_FALSE_DIR, TESTING_FALSE_DIR, split_size, all_files, False, counter)
print('split_data for false_SOURCE_DIR: Done')
split_data(true_SOURCE_DIR, TRAINING_TRUE_DIR, TESTING_TRUE_DIR, split_size, all_files, True, counter)
print('split_data for true_SOURCE_DIR: Done')
# Check that the number of images matches the expected output
print(f"\n\nThere are {len(os.listdir(TRAINING_FALSE_DIR))} images of false for training")
print(f"There are {len(os.listdir(TRAINING_TRUE_DIR))} images of true for training")
print(f"There are {len(os.listdir(TESTING_FALSE_DIR))} images of false for testing")
print(f"There are {len(os.listdir(TESTING_TRUE_DIR))} images of true for testing")

split_data for false_SOURCE_DIR: Done
split_data for true_SOURCE_DIR: Done


There are 178863 images of false for training
There are 70911 images of true for training
There are 19875 images of false for testing
There are 7875 images of true for testing


In [16]:
all_files_FALSE = listdir(TRAINING_FALSE_DIR)
all_files_FALSE[0:10]


['14079_idx5_x2151_y1401_class0.png',
 '12749_idx5_x1451_y701_class0.png',
 '14157_idx5_x1651_y301_class0.png',
 '9266_idx5_x1901_y701_class0.png',
 '13693_idx5_x1901_y2851_class0.png',
 '13022_idx5_x1501_y1001_class0.png',
 '15510_idx5_x1801_y1001_class0.png',
 '13402_idx5_x2851_y301_class0.png',
 '8867_idx5_x2501_y1301_class0.png',
 '10295_idx5_x801_y951_class0.png']

In [17]:
# TRAINING_TRUE_DIR
all_files_TRUE = listdir(TRAINING_TRUE_DIR)
all_files_TRUE[0:10]


['10286_idx5_x1251_y351_class1.png',
 '10269_idx5_x1351_y651_class1.png',
 '10307_idx5_x1351_y1801_class1.png',
 '12880_idx5_x1151_y2201_class1.png',
 '9344_idx5_x1801_y851_class1.png',
 '12749_idx5_x3201_y1351_class1.png',
 '9226_idx5_x701_y2101_class1.png',
 '15632_idx5_x1601_y1851_class1.png',
 '10308_idx5_x2051_y1951_class1.png',
 '9257_idx5_x1451_y601_class1.png']

# Training and Validation Generators

In [18]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):


  train_datagen = ImageDataGenerator(rescale=1/255)

  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=32, 
                                                      class_mode='binary',
                                                      target_size=(50, 50))

  validation_datagen = ImageDataGenerator(rescale=1/255)

  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=32, 
                                                                class_mode='binary',
                                                                target_size=(50, 50))
  ### END CODE HERE
  return train_generator, validation_generator

In [19]:
# Test your generators
TRAINING_DIR = "./IDC_regular_ps50_idx5/training/"
TESTING_DIR = "./IDC_regular_ps50_idx5/testing/"
train_generator, validation_generator = train_val_generators(TRAINING_DIR, TESTING_DIR)

Found 249774 images belonging to 2 classes.
Found 27750 images belonging to 2 classes.


In [12]:
# Used to test the model with small data.
# TRAINING_DIR_10295 = "../input/breast-histopathology-images/IDC_regular_ps50_idx5/10295/"
# TESTING_DIR_12873 = "../input/breast-histopathology-images/IDC_regular_ps50_idx5/12873/"
# train_generator, validation_generator = train_val_generators(TRAINING_DIR, TESTING_DIR)